In [29]:
#import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

#suppresses warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [43]:
tot_df = []
# for j,i in enumerate(np.arange(2016, 2021, 1)):
for j,i in enumerate(np.arange(2016, 2021, 1)):
    tot_df.append(pd.read_csv('../Data/Raw/CDC/CDCSTDcountysexrace'+ 
                              str(i) +'.csv', sep = ',', skiprows = 4))
# df = pd.concat(tot_df, axis = 0)
df = pd.read_csv('../Data/Raw/CDC/CDCSTDcountysexage2020.csv', skiprows = 4)

In [44]:
print(df.shape)
print(df.dtypes)
df = df.rename(columns = {'Cases':'cases',
                          'Rate per 100000':'ratePer100000',
                         'Indicator':'indicator',
                         'Year':'year',
                         'Geography':'geography',
                         'FIPS':'fips',
                         'Race/Ethnicity':'raceEthnicity',
                         'Sex':'sex'})
df.tail()

(180600, 8)
Indicator          object
Year               object
Geography          object
FIPS                int64
Sex                object
Age Group          object
Cases              object
Rate per 100000    object
dtype: object


,indicator,year,geography,fips,sex,Age Group,cases,ratePer100000
180595,Chlamydia,2020 (COVID-19 Pandemic),"Ziebach County, SD",46137,Female,30-34,Data suppressed,Data suppressed
180596,Chlamydia,2020 (COVID-19 Pandemic),"Ziebach County, SD",46137,Female,25-29,Data suppressed,Data suppressed
180597,Chlamydia,2020 (COVID-19 Pandemic),"Ziebach County, SD",46137,Female,20-24,Data suppressed,Data suppressed
180598,Chlamydia,2020 (COVID-19 Pandemic),"Ziebach County, SD",46137,Female,15-19,Data suppressed,Data suppressed
180599,Chlamydia,2020 (COVID-19 Pandemic),"Ziebach County, SD",46137,Female,0-14,0,0.0


In [45]:
pd.unique(df['year'])

array(['2020 (COVID-19 Pandemic)'], dtype=object)

##### Removing the 2020 (COVID-19 Pandemic) keyword to 2020

In [46]:
df.loc[df["year"].str.startswith('2020',na=False), "year"] = "2020"

In [47]:
pd.unique(df['year'])

array(['2020'], dtype=object)

##### Removing the Data Surpressed and unavailable keywords in Cases

In [50]:
# data_to_be_removed = ["cases", "ratePer100000","raceEthnicity"]
data_to_be_removed = ["cases", "ratePer100000"]

In [51]:
for col in data_to_be_removed:
    df.loc[df[col].str.contains("Data",na = False), col] = np.nan
    df[col] = df[col].str.replace(',','')

In [52]:
df.head()
pd.unique(df['year'])

array(['2020'], dtype=object)

In [53]:
# df[["year","cases","ratePer100000"]] = df[["year","cases","ratePer100000"]].apply(pd.to_numeric)

In [58]:
df.dtypes
# data.head()
df.info(memory_usage = "deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Columns: 115 entries, year to ratePer100000_Male_Primary and Secondary Syphilis_40-44
dtypes: float32(112), int32(1), object(2)
memory usage: 1.7 MB


In [57]:
for column in df:
    if df[column].dtype == 'float64':
        df[column]=pd.to_numeric(df[column], downcast='float')
    if df[column].dtype == 'int64':
        df[column]=pd.to_numeric(df[column], downcast='integer')

In [51]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
df = reduce_mem_usage(df)

Memory usage of dataframe is 5214.06 MB
Memory usage after optimization is: 10230.01 MB
Decreased by -96.2%


In [41]:
df = df.pivot_table(index=['year','geography','fips'], columns=['sex', 'indicator','raceEthnicity']).reset_index()
df.columns = df.columns.map('_'.join).str.strip('_')
df.head()

,year,geography,fips,cases_Female_Chlamydia_American Indian/Alaska Native,cases_Female_Chlamydia_Asian,cases_Female_Chlamydia_Black/African American,cases_Female_Chlamydia_Hispanic/Latino,cases_Female_Chlamydia_Multiracial,cases_Female_Chlamydia_Native Hawaiian/Other Pacific Islander,cases_Female_Chlamydia_Unknown,...,ratePer100000_Male_Gonorrhea_Multiracial,ratePer100000_Male_Gonorrhea_Native Hawaiian/Other Pacific Islander,ratePer100000_Male_Gonorrhea_White,ratePer100000_Male_Primary and Secondary Syphilis_American Indian/Alaska Native,ratePer100000_Male_Primary and Secondary Syphilis_Asian,ratePer100000_Male_Primary and Secondary Syphilis_Black/African American,ratePer100000_Male_Primary and Secondary Syphilis_Hispanic/Latino,ratePer100000_Male_Primary and Secondary Syphilis_Multiracial,ratePer100000_Male_Primary and Secondary Syphilis_Native Hawaiian/Other Pacific Islander,ratePer100000_Male_Primary and Secondary Syphilis_White
0,2019,"Abbeville County, SC",45001,0.0,0.0,39.0,1.0,0.0,0.0,26.0,...,0.0,0.0,24.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,"Acadia Parish, LA",22001,0.0,0.0,134.0,7.0,7.0,0.0,18.0,...,0.0,0.0,98.3,0.0,0.0,18.8,0.0,0.0,0.0,8.6
2,2019,"Accomack County, VA",51001,NaN,0.0,84.0,4.0,NaN,0.0,27.0,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019,"Adair County, IA",19001,0.0,0.0,NaN,0.0,NaN,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019,"Adair County, KY",21001,0.0,0.0,1.0,0.0,0.0,0.0,22.0,...,0.0,0.0,11.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
df = df.pivot_table(index=['year','geography','fips'], columns=['sex', 'indicator','Age Group']).reset_index()
df.columns = df.columns.map('_'.join).str.strip('_')
df.head()

,year,geography,fips,cases_Female_Chlamydia_0-14,cases_Female_Chlamydia_15-19,cases_Female_Chlamydia_20-24,cases_Female_Chlamydia_25-29,cases_Female_Chlamydia_30-34,cases_Female_Chlamydia_35-39,cases_Female_Chlamydia_40-44,...,ratePer100000_Male_Gonorrhea_30-34,ratePer100000_Male_Gonorrhea_35-39,ratePer100000_Male_Gonorrhea_40-44,ratePer100000_Male_Primary and Secondary Syphilis_0-14,ratePer100000_Male_Primary and Secondary Syphilis_15-19,ratePer100000_Male_Primary and Secondary Syphilis_20-24,ratePer100000_Male_Primary and Secondary Syphilis_25-29,ratePer100000_Male_Primary and Secondary Syphilis_30-34,ratePer100000_Male_Primary and Secondary Syphilis_35-39,ratePer100000_Male_Primary and Secondary Syphilis_40-44
0,2020,"Abbeville County, SC",45001,2.0,27.0,24.0,6.0,7.0,0.0,0.0,...,322.6,326.8,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020,"Acadia Parish, LA",22001,6.0,112.0,87.0,20.0,18.0,7.0,7.0,...,562.1,104.9,58.2,0.0,46.8,53.8,93.4,0.0,0.0,116.3
2,2020,"Accomack County, VA",51001,0.0,30.0,59.0,21.0,13.0,13.0,4.0,...,826.4,119.3,131.9,0.0,0.0,130.9,118.2,0.0,0.0,0.0
3,2020,"Adair County, IA",19001,0.0,5.0,1.0,4.0,1.0,1.0,0.0,...,0.0,487.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020,"Adair County, KY",21001,0.0,8.0,7.0,3.0,2.0,1.0,0.0,...,398.4,403.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
pd.unique(df['year'])

array([], dtype=int64)

In [42]:
df.to_csv("../Data/clean/CDC/disagRaceSexSTD2019_2020clean.csv",index=False)

In [36]:
pd.read_csv("../Data/clean/CDC/disagRaceSexSTD2016_2020_clean.csv")

,year,geography,fips,cases_Female_Chlamydia_American Indian/Alaska Native,cases_Female_Chlamydia_Asian,cases_Female_Chlamydia_Black/African American,cases_Female_Chlamydia_Hispanic/Latino,cases_Female_Chlamydia_Multiracial,cases_Female_Chlamydia_Native Hawaiian/Other Pacific Islander,cases_Female_Chlamydia_Unknown,...,ratePer100000_Male_Gonorrhea_Multiracial,ratePer100000_Male_Gonorrhea_Native Hawaiian/Other Pacific Islander,ratePer100000_Male_Gonorrhea_White,ratePer100000_Male_Primary and Secondary Syphilis_American Indian/Alaska Native,ratePer100000_Male_Primary and Secondary Syphilis_Asian,ratePer100000_Male_Primary and Secondary Syphilis_Black/African American,ratePer100000_Male_Primary and Secondary Syphilis_Hispanic/Latino,ratePer100000_Male_Primary and Secondary Syphilis_Multiracial,ratePer100000_Male_Primary and Secondary Syphilis_Native Hawaiian/Other Pacific Islander,ratePer100000_Male_Primary and Secondary Syphilis_White
0,2019,"Abbeville County, SC",45001,0.0,0.0,39.0,1.0,0.0,0.0,26.0,...,0.0,0.0,24.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,"Acadia Parish, LA",22001,0.0,0.0,134.0,7.0,7.0,0.0,18.0,...,0.0,0.0,98.3,0.0,0.0,18.8,0.0,0.0,0.0,8.6
2,2019,"Accomack County, VA",51001,NaN,0.0,84.0,4.0,NaN,0.0,27.0,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019,"Adair County, IA",19001,0.0,0.0,NaN,0.0,NaN,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019,"Adair County, KY",21001,0.0,0.0,1.0,0.0,0.0,0.0,22.0,...,0.0,0.0,11.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6184,2020,"Yuma County, AZ",4027,9.0,3.0,9.0,516.0,3.0,0.0,105.0,...,79.9,NaN,38.4,0.0,0.0,0.0,2.9,0.0,0.0,3.0
6185,2020,"Yuma County, CO",8125,0.0,0.0,0.0,6.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6186,2020,"Zapata County, TX",48505,0.0,0.0,0.0,25.0,0.0,0.0,16.0,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
6187,2020,"Zavala County, TX",48507,0.0,0.0,0.0,13.0,0.0,0.0,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
!ls

CDCSTD cleaning.ipynb          acs_dp05_cleaning.ipynb
Data_cleaning.ipynb            acs_ipums_cleaning.ipynb
HIV_cleaning.ipynb             google_mobility_cleaning.ipynb
NHIS.ipynb


In [62]:
!ls ../Data/clean/CDC